<h1>  This file is for the preprocessing of the multiple datasets </h1>
It includes : </br>
- Resizing the images to 224x224 (TODO) </br>
- Separate datasets into train/dev/test - 80/10/10 (TODO)

<h3> Imports and Global Variables </h3>

In [15]:
import os,sys
import numpy as np
from PIL import Image
import csv

PATH_FIRECORSE = "./../dataset/data/fire_corse/"
PATH_FIRE_SMOKE = "./../dataset/data/FIRE-SMOKE-DATASET/"
PATH_FIRE_DETECTION = "./../dataset/data/Fire-Detection-Image-Dataset/"
PATH_FIRE_KAGGLE = "./../dataset/data/Fire-Kaggle/"
PATH_WILDFIRE = "./../dataset/data/Wildfire-forest-fire/"

PATH_TARGET_FINAL = "./../dataset/data_preprocessed/"

<h3> Data exploration </h3>

In [16]:
#Finding the different image extensions
extensions_dict = {}

path_list = [PATH_FIRECORSE,PATH_FIRE_SMOKE,PATH_FIRE_DETECTION]

for p in path_list:
    for (path,dirs,files) in os.walk(p):
        if files:
            for f in files:
                f_ext = f.split('.')[-1]

                if f_ext in extensions_dict:
                    extensions_dict[f_ext] += 1
                else:
                    extensions_dict[f_ext] = 1

print(extensions_dict)

{'jpg': 3000}


<h3> Resizing images </h3>

In [17]:
def center_crop(img, new_width=None, new_height=None):        
    width, height = img.size   # Get dimensions

    left = (width - new_width)/2
    top = (height - new_height)/2
    right = (width + new_width)/2
    bottom = (height + new_height)/2

    # Crop the center of the image
    img = img.crop((left, top, right, bottom))

    return img

In [18]:
#Find the photos with max and min size, to help visualize what we are dealing with
def find_extreme_images_paths(main_path):
    MAXH,MINH,MAXW,MINW = [-1,None],[10000,None],[-1,None],[10000,None]
    for (path,dirs,files) in os.walk(main_path):
        print(path)
        if files:
            for f in files:
                img = Image.open(path+'/'+f)
                
                if img.height > MAXH[0]:
                    MAXH = [img.height,path+'/'+f]
                elif img.height < MINH[0]:
                    MINH = [img.height,path+'/'+f]
                elif img.width > MAXW[0]:
                    MAXW = [img.width,path+'/'+f]
                elif img.width < MINW[0]:
                    MINW = [img.width,path+'/'+f]

    list_path = [*set([MAXH[1],MINH[1],MAXW[1],MINW[1]])]
    return list_path

In [19]:
def resize_img(im_path):
    img = Image.open(im_path)

    img.thumbnail((225,225))

    img = center_crop(img,224,224)

    return img

In [20]:
##Test Cell
# list_h,list_w = [],[]
# extreme_images_path = find_extreme_images_paths("FIRE-SMOKE-DATASET")
# for im_path in extreme_images_path:
#     img = Image.open(im_path)

#     display(img)

#     img_tr = img.thumbnail((224,224))

#     display(img)

#     img = center_crop(img,224,224)

#     display(img)

#     im = resize_img(im_path)
#     im.save('IMAGETEST.png')


In [21]:
GLOBAL_IMAGE_COUNTER = 0

In [22]:
def process_print(current_count,size):
    if current_count%(round(size/100)) == 0:
        print("\r", end="")
        print(round(current_count/(size/100),1),"% Done",end="")
    if current_count == size:
        print("\r", end="")
        print("100.0 % Done",end="")

def resize_images(path_origin,path_target):
    global GLOBAL_IMAGE_COUNTER
    count = 0

    #Create necessary directories
    os.makedirs(path_target, exist_ok=True)

    #Resize images
    print("\nStart rezise images : "+path_origin)
    for (path,dirs,files) in os.walk(path_origin):
        
        if files:
            for f in files:
                #Print progress
                count += 1
                process_print(count,len(files))
                #Get file extension
                f_ext = f.split('.')[-1]
                if f_ext in ['jpg','png','JPG','jpeg'] and not '_nir.' in f:
                    im = resize_img(path_origin+f)
                    im.save(path_target+"image_"+str(GLOBAL_IMAGE_COUNTER)+".png")
                    GLOBAL_IMAGE_COUNTER += 1

#Ratio = (train,dev,test)
#/!\ path_origin and path_target MUST be different (Tips: create a temporary folder)
def split_images(path_origin,path_target,ratio=(8,1,1)):
    cpt = 0
    count = 0
    (n_train,n_dev,n_test) = ratio

    #Create necessary directories
    os.makedirs(path_target+"train", exist_ok=True)
    os.makedirs(path_target+"dev", exist_ok=True)
    os.makedirs(path_target+"test", exist_ok=True)

    #Split Images
    print("\nStart split images")
    for (path,dirs,files) in os.walk(path_origin):
        
        if files:
            for f in files:
                #Print progress
                count += 1
                process_print(count,len(files))

                f_ext = f.split('.')[-1]
                if f_ext in ['jpg','png','JPG','jpeg']:

                    if cpt < n_dev:
                        save_path = path_target+"dev/"+f
                    elif cpt < n_dev + n_test:
                        save_path = path_target+"test/"+f
                    elif cpt < n_dev + n_test + n_train:
                        save_path = path_target+"train/"+f

                    os.rename(path_origin+f,save_path)

                    if cpt == n_dev + n_test + n_train -1:
                        cpt = 0
                    else:
                        cpt += 1


In [23]:
def annotate(path_dir,path_file,notation):
    file_list = os.listdir(path_dir)

    annotations = []
    for fname in file_list:
        annotations.append([str(fname),int(notation)])

    file = open(path_file, 'a', newline ='')
    
    with file:
        write = csv.writer(file)
        write.writerows(annotations)
    
    file.close()


In [24]:
path_annotation = PATH_TARGET_FINAL+"annotations/annotations.csv"
path_tmp = PATH_TARGET_FINAL+"Tmp_folder/"
path_target = PATH_TARGET_FINAL+"images/"

os.makedirs(PATH_TARGET_FINAL+"annotations/", exist_ok=True)

def process_folder(path_folder,path_tmp,path_target,path_annotation,annotation):
    resize_images(path_folder,path_tmp)
    annotate(path_tmp,path_annotation,int(annotation))
    split_images(path_tmp,path_target)


#Fire-Detection
#/!\ You need to create and manually put the raw images in Fire_images and Normal_images
path_dir = PATH_FIRE_DETECTION
process_folder(path_dir+"Fire_images/",path_tmp,path_target,path_annotation,1)
process_folder(path_dir+"Normal_images/",path_tmp,path_target,path_annotation,0)

#FireCorse
path_dir = PATH_FIRECORSE
process_folder(path_dir,path_tmp,path_target,path_annotation,1)

#FIRE-SMOKE
path_dir = PATH_FIRE_SMOKE
process_folder(path_dir+"Test/Fire/",path_tmp,path_target,path_annotation,1)
process_folder(path_dir+"Test/Neutral/",path_tmp,path_target,path_annotation,0)
process_folder(path_dir+"Test/Smoke/",path_tmp,path_target,path_annotation,0)
process_folder(path_dir+"Train/Fire/",path_tmp,path_target,path_annotation,1)
process_folder(path_dir+"Train/Neutral/",path_tmp,path_target,path_annotation,0)
process_folder(path_dir+"Train/Smoke/",path_tmp,path_target,path_annotation,0)

#Fire KAGGLE 
path_dir = PATH_FIRE_KAGGLE
process_folder(path_dir+"fire/",path_tmp,path_target,path_annotation,1)
process_folder(path_dir+"no_fire/",path_tmp,path_target,path_annotation,0)

#Wildfire
path_dir = PATH_WILDFIRE
process_folder(path_dir,path_tmp,path_target,path_annotation,1)

os.rmdir(path_tmp)


Start rezise images : ./../dataset/data/Fire-Detection-Image-Dataset/Fire_images/

Start split images

Start rezise images : ./../dataset/data/Fire-Detection-Image-Dataset/Normal_images/

Start split images

Start rezise images : ./../dataset/data/fire_corse/

Start split images

Start rezise images : ./../dataset/data/FIRE-SMOKE-DATASET/Test/Fire/
100.0 % Done
Start split images
100.0 % Done
Start rezise images : ./../dataset/data/FIRE-SMOKE-DATASET/Test/Neutral/
100.0 % Done
Start split images
100.0 % Done
Start rezise images : ./../dataset/data/FIRE-SMOKE-DATASET/Test/Smoke/
100.0 % Done
Start split images
100.0 % Done
Start rezise images : ./../dataset/data/FIRE-SMOKE-DATASET/Train/Fire/
100.0 % Done
Start split images
100.0 % Done
Start rezise images : ./../dataset/data/FIRE-SMOKE-DATASET/Train/Neutral/
100.0 % Done
Start split images
100.0 % Done
Start rezise images : ./../dataset/data/FIRE-SMOKE-DATASET/Train/Smoke/
100.0 % Done
Start split images
100.0 % Done
Start rezise imag

In [25]:
path_annotation = PATH_TARGET_FINAL+"annotations/"
path_data = PATH_TARGET_FINAL+"images/"

dir_list = ["dev","train","test"]
ids = {}
reversed_ids = {}
header = ["name","label"]
annotations_different_csv = {"dev":[],"test":[],"train":[]}

for dir in dir_list:
    (path,dirs,files) = list(os.walk(path_data+dir))[0]
    ids[dir] = files

# Création du dictionnaire inversé : nom de l'image -> nom du dossier
for dir in ids:
    list_name = ids[dir]
    for name in list_name:
        reversed_ids[name] = dir

# Lecture du fichier de toutes les annotations
file = open(path_annotation+"annotations.csv", 'r', newline ='')   
with file:
    for row in file:
        row_list = row.split(',')
        im_name = row_list[0]
        row_list[1]=int(row_list[1])
        dir = reversed_ids[im_name]
        annotations_different_csv[dir].append(row_list)

file.close()

for dir in dir_list:
    file = open(path_annotation+"annotations_"+dir+".csv", "w+",newline="")
    
    with file:
        write = csv.writer(file)
        write.writerows(annotations_different_csv[dir])
    
    file.close()
    
